# Get Dublin region EV charge points

... by imitating the POST request that the user of ESB's web map makes to Google Maps. 

ESB Map: https://esb.ie/ecars/charge-point-map

__Note__: To find out what information the ESB's web map sends to Google Maps (i.e. a POST request) to get charge point coordinates to display;
- `firefox` inspect network tool to observe the POST request
- Right-click copy `curl` command
- Paste to https://curl.trillworks.com/ to convert this request to `python` code which generates the `headers` and `data` inputs below

In [11]:
import json
import requests

In [12]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0',
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.5',
    'Content-Type': 'application/json',
    'Origin': 'https://esb.ie',
    'DNT': '1',
    'Connection': 'keep-alive',
    'Referer': 'https://esb.ie/ecars/DrivezMap.html?53.3252596,-6.3569407,9z',
    'Sec-GPC': '1',
}

data = '{"filterByBounds":{"northEastLat":53.67162057190402,"northEastLng":-4.747443601953134,"southWestLat":52.850188497467684,"southWestLng":-7.960944578515634}}'

response = requests.post('https://myaccount.esbecars.com/stationFacade/findSitesInBounds', headers=headers, data=data)


In [13]:
response.json()

{'errors': [],
 'success': True,
 'data': ['java.util.ArrayList',
  [{'@class': 'com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp',
    'id': 80,
    'deleted': False,
    'dirty': False,
    'managed': True,
    'fast': False,
    'serviceProviderLinkedCheckerId': 80,
    'siteId': 80,
    'dn': 'The Square Shopping Centre, Belgard Road (R113), Tallaght, Dublin 24',
    'latitude': 53.285621,
    'longitude': -6.369887,
    'ss': 'AVAILABLE',
    'im': True,
    'ns': 2,
    'hg': False,
    'imn': False,
    'mfr': False,
    'isn': False,
    'isf': False,
    'scs': 'SEMI_FAST'},
   {'@class': 'com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp',
    'id': 1290,
    'deleted': False,
    'dirty': False,
    'managed': False,
    'fast': True,
    'serviceProviderLinkedCheckerId': 1290,
    'siteId': 1290,
    'dn': 'Windsor Motors Belgard (Nissan), Belgard Road, Dublin 24',
    'latitude': 53.295,
    'longitude': -6.3694,
    'ss': 'UNKNOWN',
    'im': False,
    'ns': 1,
    'hg'

In [14]:
with open("response.json", "w") as json_file:
    json.dump(response.json(), json_file)

In [16]:
with open("response.json", "r") as json_file:
    response_data = json.load(json_file)

# Convert `raw_locations` response to a `GeoDataFrame`

... so can use `GeoPandas` to re-plot all Ireland EV charge points 

In [21]:
import pandas as pd
import geopandas as gpd

In [18]:
response_data = response.json()["data"][1]

In [19]:
response_data

[{'@class': 'com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp',
  'id': 80,
  'deleted': False,
  'dirty': False,
  'managed': True,
  'fast': False,
  'serviceProviderLinkedCheckerId': 80,
  'siteId': 80,
  'dn': 'The Square Shopping Centre, Belgard Road (R113), Tallaght, Dublin 24',
  'latitude': 53.285621,
  'longitude': -6.369887,
  'ss': 'AVAILABLE',
  'im': True,
  'ns': 2,
  'hg': False,
  'imn': False,
  'mfr': False,
  'isn': False,
  'isf': False,
  'scs': 'SEMI_FAST'},
 {'@class': 'com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp',
  'id': 1290,
  'deleted': False,
  'dirty': False,
  'managed': False,
  'fast': True,
  'serviceProviderLinkedCheckerId': 1290,
  'siteId': 1290,
  'dn': 'Windsor Motors Belgard (Nissan), Belgard Road, Dublin 24',
  'latitude': 53.295,
  'longitude': -6.3694,
  'ss': 'UNKNOWN',
  'im': False,
  'ns': 1,
  'hg': False,
  'imn': False,
  'mfr': False,
  'isn': False,
  'isf': True,
  'scs': 'FAST'},
 {'@class': 'com.driivz.appserver.bl.dto.imp.S

In [23]:
ev_chargepoints_df = pd.DataFrame(response_data)

How to convert a `DataFrame` with `lat` and `long` to a `GeoDataFrame`: https://geopandas.readthedocs.io/en/latest/gallery/create_geopandas_from_pandas.html

In [24]:
ev_chargepoints_gdf = gpd.GeoDataFrame(
    ev_chargepoints_df,
    geometry=gpd.points_from_xy(
        ev_chargepoints_df["longitude"],
        ev_chargepoints_df["latitude"],
    ),
    crs="epsg:4326"
)

In [25]:
ev_chargepoints_gdf

,@class,id,deleted,dirty,managed,fast,serviceProviderLinkedCheckerId,siteId,dn,latitude,...,ss,im,ns,hg,imn,mfr,isn,isf,scs,geometry
0,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,80,False,False,True,False,80,80,"The Square Shopping Centre, Belgard Road (R113...",53.285621,...,AVAILABLE,True,2,False,False,False,False,False,SEMI_FAST,POINT (-6.36989 53.28562)
1,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,1290,False,False,False,True,1290,1290,"Windsor Motors Belgard (Nissan), Belgard Road,...",53.295000,...,UNKNOWN,False,1,False,False,False,False,True,FAST,POINT (-6.36940 53.29500)
2,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,523,False,False,True,False,523,523,"Citywest Shopping Centre Car Park, Citywest Dr...",53.283222,...,CHARGING,True,2,False,False,False,False,False,SEMI_FAST,POINT (-6.42171 53.28322)
3,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,2,False,False,True,True,2,2,"Circle K / Topaz Service Station, Templeville ...",53.304400,...,AVAILABLE,True,1,False,False,False,False,True,FAST,POINT (-6.31380 53.30440)
4,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,1302,False,False,False,False,1302,1302,"Lidl Rathfarnham, 13 Grange Road (R822), Dubli...",53.277425,...,UNKNOWN,False,1,False,False,False,False,False,SEMI_FAST,POINT (-6.26585 53.27743)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,25,False,False,True,True,25,25,"Circle K - Topaz Service Station, Castlemaine ...",53.420872,...,AVAILABLE,True,2,False,False,False,False,True,FAST,POINT (-7.92666 53.42087)
197,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,133,False,False,True,False,133,133,"Ballymahon Road, Athlone, Westmeath",53.424683,...,AVAILABLE,True,1,False,False,False,False,False,SEMI_FAST,POINT (-7.93455 53.42468)
198,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,384,False,False,True,False,384,384,"Irish Rail Train Station, Southern Station Roa...",53.427551,...,AVAILABLE,True,1,False,False,False,False,False,SEMI_FAST,POINT (-7.93842 53.42755)
199,com.driivz.appserver.bl.dto.imp.SiteMapDataDtoImp,134,False,False,True,False,134,134,"Castle Street, Athlone, Westmeath",53.422802,...,AVAILABLE,True,1,False,False,False,False,False,SEMI_FAST,POINT (-7.94274 53.42280)


# Get Dublin EV Charge Points

... as bounding box used by `requests` above includes outside of Dublin data

In [79]:
import contextily as cx

In [28]:
dublin_boundary = gpd.read_file("dublin_boundary_epsg_4326")

In [29]:
dublin_ev_chargepoints = gpd.sjoin(ev_chargepoints_gdf, dublin_boundary)

In [ ]:
dublin_ev_chargepoints.to_file("../data/dublin_ev_chargepoint_locations")